In [16]:
import numpy as np
from math import exp, sqrt
import matplotlib.pyplot as plt
import BayesODE as bo
from BayesODE.Tests.root_gen import root_gen
from BayesODE.utils import mvCond
from BayesODE.kalman_initial_draw import kalman_initial_draw
from BayesODE.kalman_ode_higher import kalman_ode_higher
from BayesODE.higher_mvCond import higher_mvCond
from BayesODE.Tests.test_exp_integrate import cov_yy_ex
from BayesODE.cov_car import cov_car
from BayesODE.kalman_filter import kalman_filter
from BayesODE.kalman_smooth import kalman_smooth
from BayesODE.filter_update_full import filter_update_full
from pykalman import standard as pks
import scipy as sc

In [ ]:
from math import sin, cos
def chk_F(y_t, t):
    return sin(2*t) - y_t[0] #X^{2} = sin(2t) - X

def chk_exact_x(t):
    return (-3*cos(t) + 2*sin(t) - sin(2*t))/3

def chk_exact_x1(t):
    return (-2*cos(2*t) + 3*sin(t) + 2*cos(t))/3

def chk_exact_x2(t):
    return (4*sin(2*t) - 2*sin(t) + 3*cos(t))/3


# Test Against Pykalman

In [3]:
#Define parameters
N = 99
q = 2
p = q+2

delta_t = np.array([1/N])
r0 = 0.5
sigma = 0.001
roots = root_gen(r0, p) #Generate roots to draw x^{(3)}_0
a = np.array([0,0,1])
x0 = np.array([-1,0,0])
x0 = np.array([-1,0,chk_F(x0, 0)]) #Initial state

Y0 = kalman_initial_draw(roots, sigma, x0, p)
A, V = higher_mvCond(delta_t, roots, sigma) 
lam = np.zeros(p)
b = lam - A.dot(lam)
fun = chk_F

In [4]:
# notation consistent with pykalman package
n_dim_obs = 1
n_dim_state = len(Y0)
n_timesteps = N+1

# allocate memory
us = np.zeros((n_timesteps,n_dim_obs)) 

# var(us_n | y_n), to be determined during the interrogation process
sig2 = np.zeros((n_timesteps, n_dim_obs, n_dim_obs))

# solution process
# forward mean and variance.
mu = np.zeros((n_timesteps, n_dim_state)) # E[y_n | us_0:n]
# var(y_n | us_0:n)
Sigma = np.zeros((n_timesteps, n_dim_state, n_dim_state))

#a padde with 0s
p = len(b)
q = len(a) - 1
a_star = np.pad(a, (0, p-q-1), 'constant', constant_values=(0,0))

# argumgents for kalman_filter and kalman_smooth
D = np.array([a_star])
e = np.array([0.])
F = sig2
mu_currs = mu
Sigma_currs = Sigma
mu_preds = np.zeros((n_timesteps, n_dim_state))
Sigma_preds = np.zeros((n_timesteps, n_dim_state, n_dim_state))

# arguments to use low-level pykalman functions
observations = us
observation_matrix = np.array([a_star])
observation_offset = np.array([0.])
observation_covariances = sig2 # multidimensional
transition_matrix = A
transition_offset = b
transition_covariance = V # single dimensional
filtered_state_means = mu
filtered_state_covariances = Sigma
predicted_state_means = np.zeros((n_timesteps, n_dim_state))
predicted_state_covariances = np.zeros((n_timesteps, n_dim_state, n_dim_state))

# initialize things
mu[0] = Y0
us[0] = Y0.dot(a_star)
mu_preds[0] = predicted_state_means[0] = mu[0]
Sigma_preds[0] = predicted_state_covariances[0] = Sigma[0]

# forward pass: merging pks._filter to accommodate multiple
# observation_covariances
# calculate mu_tt = E[y_t | us_0:t-1] and
# Sigma_tt = var(y_t | us_0:t-1)

for t in range(N):
    mu_tt = np.dot(A, mu[t]) + b
    Sigma_tt = np.linalg.multi_dot([A, Sigma[t], A.T]) + V #A*Sigma[t]*A.T + V 
    sig2[t+1] = np.linalg.multi_dot([a_star, Sigma_tt, a_star.T]) # new observation_covariance
    I_tt = np.random.multivariate_normal(np.zeros(p), np.eye(p))
    D_tt = np.linalg.cholesky(np.absolute(Sigma_tt, where=np.eye(p, dtype=bool)))
    Yt1 = mu_tt + D_tt.dot(I_tt) #Y_{t+1} ~ p(Y_{t+1} | Y_t)
    us[t+1] = fun(Yt1,(t+1)/N) #new observation (u_{t+1})
    (mu_preds[t+1], Sigma_preds[t+1], mu_currs[t+1], Sigma_currs[t+1]) = (
        kalman_filter(mu_curr = mu_currs[t],
                    Sigma_curr = Sigma_currs[t],
                    u_star = us[t+1],
                    A = A,
                    b = b,
                    V = V,
                    D = D,
                    e = e,
                    F = F[t+1]
                    )
    )
    (predicted_state_means[t+1], predicted_state_covariances[t+1],
         _, filtered_state_means[t+1],
         filtered_state_covariances[t+1]) = (
             filter_update_full(filtered_state_mean = filtered_state_means[t],
                                filtered_state_covariance = filtered_state_covariances[t],
                                observation = observations[t+1],
                                transition_matrix = transition_matrix,
                                transition_offset = transition_offset,
                                transition_covariance = transition_covariance,
                                observation_matrix = observation_matrix,
                                observation_offset = observation_offset,
                                observation_covariance = observation_covariances[t+1])
         )
 # backward pass
(Y_tt, mu_smooth, Sigma_smooth) = (
    kalman_smooth(
        A = A, 
        mu_currs = mu_currs,
        Sigma_currs = Sigma_currs, 
        mu_preds = mu_preds,
        Sigma_preds = Sigma_preds
    )
)
# backward pass
(smoothed_state_means, smoothed_state_covariances, _) = (
    pks._smooth(
        transition_matrix, filtered_state_means,
        filtered_state_covariances, predicted_state_means,
        predicted_state_covariances
    )
)

Yn_mean = mu_smooth
Yn_var = Sigma_smooth

Yn_pykalman_mean = smoothed_state_means
Yn_pykalman_var = smoothed_state_covariances

In [5]:
print("Means are equal:{}".format(np.allclose(Yn_mean, Yn_pykalman_mean)))
print("Covariances are equal:{}".format(np.allclose(Yn_var, Yn_pykalman_var)))

Means are equal:True
Covariances are equal:True


# MVGSS

To differentiate the notation used in the general model and the state-space model. I will use double letter for the general model. For example, we have

$AA_n =\begin{bmatrix} A_n & 0 \\ D_nA_n & 0 \end{bmatrix}$, 
$bb_n = \begin{bmatrix} b_n \\ D_nb_n + e_n \end{bmatrix}$, and 
$CC_n = \begin{bmatrix} C_n & 0 \\ D_nC_n & F_n \end{bmatrix}$.

We also have 

$bb_0 = \begin{bmatrix} b_0 \\ D_0b_0 \end{bmatrix}$ and 
$CC_0 = \begin{bmatrix} C_0 & 0 \\ D_0C_0 & F_0 \end{bmatrix}$

where 

$A_n = A$, $b_0 = \lambda$, $b_n = \lambda - A \lambda$, $D_n = a_\star'$, $e_n = 0$, $F_0 = 0$ and $F_n^2 = \sigma_n^2 = a_\star'\Sigma_{n|n-1}a_\star$.

In [7]:
N = 3
tseq = np.linspace(0,1,N)
XX = np.zeros(N)
for t in range(N):
    XX[t] = chk_exact_x2(tseq[t])
    

In [8]:
N = 2
q = 2
p = q+2
delta_t = np.array([1/N])
r0 = 0.5
sigma = 0.001
roots = root_gen(r0, p) #Generate roots to draw x^{(3)}_0
a = np.array([0,0,1])
Z0 = np.array([-1,0,0])
Z0 = np.array([-1,0,chk_F(x0, 0)]) #Initial state

Y0 = kalman_initial_draw(roots, sigma, x0, p)
A, V = higher_mvCond(delta_t, roots, sigma)
lam = np.zeros(p)
b = lam - A.dot(lam)

Yn_chk_mean, Yn_chk_var, sigma_preds = kalman_ode_higher(chk_F, Y0, N, A, b, V, a, True)

In [9]:
a_star = np.pad(a, (0, p-q-1), 'constant', constant_values=(0,0)) #D_n
e0 = 0
F0 = 0

#bb0, CC0
bb0 = np.append(lam, np.array(a_star.dot(lam))) 
V_inf = cov_car([], roots, sigma, v_infinity=True) 
C = np.linalg.cholesky(V_inf)
CC0 = np.zeros((p+1, p+1))
CC0[0:p,0:p] = C
CC0[p,0:p] = a_star.dot(C)

#AA, bb, CC
AAn = np.zeros((p+1,p+1))
AAn[0:p,0:p] = A
AAn[p,0:p] = a_star.dot(A)
AA = np.dstack([AAn]*(N+1))
AA[:,:,0] = np.nan
bbn = np.append(b, a_star.dot(b)) 
bb = np.repeat(bbn[:,np.newaxis],N+1,axis=1)
bb[:,0] = bb0
CC = np.zeros((p+1, p+1, N+1))
CC[:,:,0] = CC0

for i in range(1,N+1):
    CCn = np.zeros((p+1, p+1))
    Cn = np.linalg.cholesky(V)
    CCn[0:p,0:p] = Cn
    CCn[p,0:p] = a_star.dot(Cn)
    CCn[p,p] = np.sqrt(np.linalg.multi_dot([a_star, sigma_preds[i], a_star]))
    CC[:,:,i] = CCn


In [10]:
AA[:,:,1]

array([[ 0.98918226,  0.46415456,  0.09019494,  0.00682349,  0.        ],
       [-0.06852673,  0.77006529,  0.26983804,  0.02640423,  0.        ],
       [-0.26517159, -0.91642356,  0.01813653,  0.02299301,  0.        ],
       [-0.23091345, -1.00352665, -1.57120889, -0.19681802,  0.        ],
       [-0.26517159, -0.91642356,  0.01813653,  0.02299301,  0.        ]])

In [11]:
CC[:,:,0]

array([[ 4.18968686e-05,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-6.71270712e-19,  2.65318878e-05,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-1.68017585e-05, -5.41338888e-18,  4.62135610e-05,
         0.00000000e+00,  0.00000000e+00],
       [-3.83019171e-18, -9.11353286e-05, -2.49553366e-17,
         2.31264820e-04,  0.00000000e+00],
       [-1.68017585e-05, -5.41338888e-18,  4.62135610e-05,
         0.00000000e+00,  0.00000000e+00]])

In [12]:
def mvGSS(AA, bb, CC):
    d,veclen = bb.shape
    N = veclen-1
    D = d*N + d
    An_m = np.zeros((d,d,N+1,N+1))
    for n in range(N+1):
        for m in range(N+1):
            if m>n:
                An_m[:,:,n,m] = np.eye(d)
            elif n==m:
                An_m[:,:,n,m] = AA[:,:,n]
            else:
                diff = n-m
                A_diff = AA[:,:,m]
                for i in range(diff):
                    A_diff = np.matmul(AA[:,:,m+i+1],A_diff)
                An_m[:,:,n,m] = A_diff
    L = np.zeros((D,D))
    mean_Y = np.zeros(D)
    for n in range(N+1):
        for m in range(n,N+1):
            if n == N:
                L[m*d:m*d+d,n*d:n*d+d] = np.matmul(np.eye(d), CC[:,:,n])
            else:
                L[m*d:m*d+d,n*d:n*d+d] = np.matmul(An_m[:,:,m,n+1], CC[:,:,n])
        for l in range(n):
            mean_Y[n*d:n*d+d] = mean_Y[n*d:n*d+d] + An_m[:,:,n,l+1].dot(bb[:,l])
    LL = np.matmul(L,L.T)
    var_Y = LL
    return An_m, mean_Y, var_Y

In [13]:
An_m, mean_Y, var_Y = mvGSS(AA,bb,CC)

In [14]:
Yn_chk_var

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[ 3.59994777e-12,  1.71577000e-11,  2.34454218e-11,
         -1.82105683e-10],
        [ 1.71577000e-11,  8.90402979e-11,  1.69561843e-10,
         -9.13769494e-10],
        [ 2.34454218e-11,  1.69561843e-10,  7.74145113e-10,
          2.27709711e-10],
        [-1.82105683e-10, -9.13769494e-10,  2.27709711e-10,
          5.26572958e-08]],

       [[ 6.34214785e-11,  1.18226978e-10, -1.07113858e-11,
         -4.87384266e-10],
        [ 1.18226978e-10,  2.89544479e-10,  1.42005142e-10,
         -1.75011815e-09],
        [-1.07113858e-11,  1.42005142e-10,  8.57309816e-10,
          8.99686195e-11],
        [-4.87384266e-10, -1.75011815e-

In [15]:
icond = np.array([False, False, False, False, True]*(N+1))
R,s,T = mvCond(mean_Y, var_Y, icond)

In [17]:
mean_Y[icond]

array([0., 0., 0.])

In [18]:
Yn_chk_mean

array([[-1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        -1.56166971e-04],
       [-8.86826849e-01,  4.47118914e-01,  9.38149364e-01,
        -1.28766806e-01],
       [-5.61839301e-01,  8.27331702e-01,  6.12046028e-01,
        -1.56179179e+00]])

In [19]:
R.dot(XX)

array([-0.55420287, -0.44324814,  1.        ,  1.44794256, -0.64421652,
        0.08842505,  1.0925614 , -0.45892123, -0.48453198,  0.51120902,
        0.61980702, -1.47812653])

In [20]:
mean_Y.shape

(15,)

In [21]:
R

array([[-3.00045374e-01, -9.84891492e-02, -7.44328036e-02],
       [-1.42207202e-02, -1.68142363e-01, -1.22982550e-01],
       [ 1.00000000e+00,  1.61985982e-17, -2.90598591e-17],
       [-8.91897693e-02,  9.19429422e-01, -6.24463389e-03],
       [-2.16875225e-01, -1.56333284e-01, -1.38214568e-01],
       [ 2.58011532e-01, -7.29905810e-03, -1.32014931e-01],
       [ 4.48386891e-02,  6.03400401e-01,  2.85756939e-02],
       [-1.33906590e+00,  3.78817169e-02,  6.85150352e-01],
       [-9.67557796e-02, -1.04240142e-01, -1.78448548e-01],
       [ 2.12520754e-01,  1.62532523e-01,  2.15197783e-02],
       [-1.09348214e-01,  3.13830819e-02,  5.67612331e-01],
       [ 1.11729428e-03, -9.16937882e-01,  5.13080524e-02]])